In [1]:
#Set environment options
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)

### Set environmental variables and install libraries

# Call nba_api for all NBA games in a given season

In [2]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import time

#get nba team data including id's
nba_teams = pd.DataFrame(teams.get_teams())
#Create list of NBA team id's
team_ids = nba_teams['id'].tolist()

#season data should be pulled for 
s = '2022-23'


#function to call api and get all NBA games for a given season
def get_all_games(team__ids,season):
    team_season = []
    for i in team_ids:
        games = leaguegamefinder.LeagueGameFinder(team_id_nullable = i,season_nullable = season,season_type_nullable ='Regular Season').get_data_frames()[0]
        team_season.append(games)
        nba_season = pd.concat(team_season)
        time.sleep(1)
    return(nba_season)

nba_season = get_all_games(team_ids,s)

In [3]:
nba_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 12
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          350 non-null    object 
 1   TEAM_ID            350 non-null    int64  
 2   TEAM_ABBREVIATION  350 non-null    object 
 3   TEAM_NAME          350 non-null    object 
 4   GAME_ID            350 non-null    object 
 5   GAME_DATE          350 non-null    object 
 6   MATCHUP            350 non-null    object 
 7   WL                 343 non-null    object 
 8   MIN                350 non-null    int64  
 9   PTS                350 non-null    int64  
 10  FGM                350 non-null    int64  
 11  FGA                350 non-null    int64  
 12  FG_PCT             350 non-null    float64
 13  FG3M               350 non-null    int64  
 14  FG3A               350 non-null    int64  
 15  FG3_PCT            350 non-null    float64
 16  FTM                350 non-

Get team stats for specific team

In [14]:
# subset TEAM_ABBREVIATION = 'DAL'
NBA_seasons_DAL = nba_season[nba_season.MATCHUP.str.contains('DAL',regex=False)]

In [3]:
#function to combine matchup data
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        #pass
        return None
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    

In [20]:
# Combine the game rows into one. By default, the home team will be TEAM_A .
game_df = combine_team_games(nba_season,keep_method='home')
#Sort the dataframe by game date
game_df = game_df.sort_values(by= 'GAME_DATE')
#add total points for game together
game_df['TOTAL_GAME_PTS'] = game_df['PTS_A'] + game_df['PTS_B']

game_df.head(20)

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A,TEAM_ID_B,TEAM_ABBREVIATION_B,TEAM_NAME_B,MATCHUP_B,WL_B,MIN_B,PTS_B,FGM_B,FGA_B,FG_PCT_B,FG3M_B,FG3A_B,FG3_PCT_B,FTM_B,FTA_B,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B,TOTAL_GAME_PTS
113,22022,1610612738,BOS,Boston Celtics,0022200001,2022-10-18,BOS vs. PHI,W,240,126,46,82,0.561,12,35,0.343,22,28,0.786,6,30,36,24,8,3,10,24,9.0,1610612755,PHI,Philadelphia 76ers,PHI @ BOS,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0,243
405,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,45,99,0.455,16,45,0.356,17,23,0.739,11,37,48,31,11,4,18,23,14.0,1610612747,LAL,Los Angeles Lakers,LAL @ GSW,L,241,109,40,94,0.426,10,40,0.250,19,25,0.760,9,39,48,23,12,4,21,18,-14.0,232
637,22022,1610612750,MIN,Minnesota Timberwolves,0022200010,2022-10-19,MIN vs. OKC,W,241,115,39,94,0.415,10,38,0.263,27,33,0.818,14,41,55,25,6,5,14,21,7.0,1610612760,OKC,Oklahoma City Thunder,OKC @ MIN,L,241,108,38,99,0.384,14,45,0.311,18,22,0.818,16,41,57,22,6,6,13,26,-7.0,223
258,22022,1610612748,MIA,Miami Heat,0022200007,2022-10-19,MIA vs. CHI,L,240,108,37,80,0.463,12,28,0.429,22,28,0.786,8,38,46,21,6,2,19,24,-8.0,1610612741,CHI,Chicago Bulls,CHI @ MIA,W,240,116,41,93,0.441,13,36,0.361,21,28,0.750,12,34,46,23,12,6,14,26,8.0,224
702,22022,1610612763,MEM,Memphis Grizzlies,0022200009,2022-10-19,MEM vs. NYK,W,266,115,42,108,0.389,17,50,0.340,14,23,0.609,20,44,64,27,6,7,13,17,3.0,1610612752,NYK,New York Knicks,NYK @ MEM,L,265,112,44,109,0.404,9,37,0.243,15,18,0.833,18,36,54,28,7,5,12,26,-3.0,227
726,22022,1610612765,DET,Detroit Pistons,0022200003,2022-10-19,DET vs. ORL,W,239,113,40,94,0.426,14,38,0.368,19,24,0.792,12,29,41,31,11,4,12,21,4.0,1610612753,ORL,Orlando Magic,ORL @ DET,L,240,109,42,86,0.488,11,30,0.367,14,19,0.737,10,38,48,21,5,5,18,24,-4.0,222
57,22022,1610612737,ATL,Atlanta Hawks,0022200005,2022-10-19,ATL vs. HOU,W,240,117,45,90,0.500,7,25,0.280,20,24,0.833,4,34,38,30,12,5,9,18,10.0,1610612745,HOU,Houston Rockets,HOU @ ATL,L,240,107,42,98,0.429,9,35,0.257,14,15,0.933,15,39,54,25,4,3,15,20,-10.0,224
162,22022,1610612761,TOR,Toronto Raptors,0022200008,2022-10-19,TOR vs. CLE,W,240,108,36,86,0.419,13,30,0.433,23,32,0.719,12,29,41,19,15,4,13,23,3.0,1610612739,CLE,Cleveland Cavaliers,CLE @ TOR,L,242,105,38,77,0.494,10,28,0.357,19,23,0.826,6,32,38,25,5,5,17,21,-3.0,213
358,22022,1610612762,UTA,Utah Jazz,0022200012,2022-10-19,UTA vs. DEN,W,242,123,42,83,0.506,16,38,0.421,23,31,0.742,11,32,43,27,10,1,19,21,21.0,1610612743,DEN,Denver Nuggets,DEN @ UTA,L,238,102,40,83,0.482,5,22,0.227,17,18,0.944,10,25,35,21,10,3,21,23,-21.0,225
310,22022,1610612756,PHX,Phoenix Suns,0022200013,2022-10-19,PHX vs. DAL,W,240,107,40,85,0.471,8,22,0.364,19,22,0.864,8,32,40,25,4,5,12,29,2.0,1610612742,DAL,Dallas Mavericks,DAL @ PHX,L,240,105,35,75,0.467,14,35,0.400,21,34,0.618,5,35,40,17,6,4,12,22,-2.0,212


# Explore NBA_API End Points
End point exploration documentation below starts with endpoint name and datasets that can be returned. 
*************************************************************************************************************************************** 
***************************************************************************************************************************************


# GameRotation

* gamerotation_away:  
['GAME_ID', 'TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'PERSON_ID', 'PLAYER_FIRST', 'PLAYER_LAST', 'IN_TIME_REAL', 'OUT_TIME_REAL', 'PLAYER_PTS', 'PT_DIFF', 'USG_PCT']

* gamerotation_home:  
['GAME_ID', 'TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'PERSON_ID', 'PLAYER_FIRST', 'PLAYER_LAST', 'IN_TIME_REAL', 'OUT_TIME_REAL', 'PLAYER_PTS', 'PT_DIFF', 'USG_PCT']

In [10]:
#home and away player rotations by game
from nba_api.stats.endpoints import gamerotation

gr = gamerotation.GameRotation(game_id= '0022200149')

gamerotation_away = gr.get_data_frames()[0]
gamerotation_home = gr.get_data_frames()[1]

# BoxScoreSummary
* game_summary:  
['GAME_DATE_EST', 'GAME_SEQUENCE', 'GAME_ID', 'GAME_STATUS_ID', 'GAME_STATUS_TEXT', 'GAMECODE', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'LIVE_PERIOD', 'LIVE_PC_TIME', 'NATL_TV_BROADCASTER_ABBREVIATION', 'LIVE_PERIOD_TIME_BCAST', 'WH_STATUS']

* other_stats:  
['LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PTS_PAINT', 'PTS_2ND_CHANCE', 'PTS_FB', 'LARGEST_LEAD', 'LEAD_CHANGES', 'TIMES_TIED', 'TEAM_TURNOVERS', 'TOTAL_TURNOVERS', 'TEAM_REBOUNDS', 'PTS_OFF_TO']

* officials:  
['OFFICIAL_ID', 'FIRST_NAME', 'LAST_NAME', 'JERSEY_NUM']

* inactive_players:  
['PLAYER_ID', 'FIRST_NAME', 'LAST_NAME', 'JERSEY_NUM', 'TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'TEAM_ABBREVIATION']

* game_info:  
['GAME_DATE', 'ATTENDANCE', 'GAME_TIME']

* line_score:  
['GAME_DATE_EST', 'GAME_SEQUENCE', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY_NAME', 'TEAM_NICKNAME', 'TEAM_WINS_LOSSES', 'PTS_QTR1', 'PTS_QTR2', 'PTS_QTR3', 'PTS_QTR4', 'PTS_OT1', 'PTS_OT2', 'PTS_OT3', 'PTS_OT4', 'PTS_OT5', 'PTS_OT6', 'PTS_OT7', 'PTS_OT8', 'PTS_OT9', 'PTS_OT10', 'PTS']

* last_meeting:  
['GAME_ID', 'LAST_GAME_ID', 'LAST_GAME_DATE_EST', 'LAST_GAME_HOME_TEAM_ID', 'LAST_GAME_HOME_TEAM_CITY', 'LAST_GAME_HOME_TEAM_NAME', 'LAST_GAME_HOME_TEAM_ABBREVIATION', 'LAST_GAME_HOME_TEAM_POINTS', 'LAST_GAME_VISITOR_TEAM_ID', 'LAST_GAME_VISITOR_TEAM_CITY', 'LAST_GAME_VISITOR_TEAM_NAME', 'LAST_GAME_VISITOR_TEAM_CITY1', 'LAST_GAME_VISITOR_TEAM_POINTS']

* season_series:  
['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'GAME_DATE_EST', 'HOME_TEAM_WINS', 'HOME_TEAM_LOSSES', 'SERIES_LEADER']

* available_video:  
['GAME_ID', 'VIDEO_AVAILABLE_FLAG', 'PT_AVAILABLE', 'PT_XYZ_AVAILABLE', 'WH_STATUS', 'HUSTLE_STATUS', 'HISTORICAL_STATUS']

In [27]:
from nba_api.stats.endpoints import boxscoresummaryv2

bxv2_call = boxscoresummaryv2.BoxScoreSummaryV2(game_id='0022200149')

game_summary = bxv2_call.get_data_frames()[0]
other_stats = bxv2_call.get_data_frames()[1]
officials = bxv2_call.get_data_frames()[2]
inactive_players = bxv2_call.get_data_frames()[3]
game_info = bxv2_call.get_data_frames()[4]
line_score = bxv2_call.get_data_frames()[5]
last_meeting = bxv2_call.get_data_frames()[6]
season_series = bxv2_call.get_data_frames()[7]
available_video = bxv2_call.get_data_frames()[8]


# BoxScoreAdvancedV2

* player_stats:  
['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID', 'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE']

* team_stats:  
['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE']

In [31]:
from nba_api.stats.endpoints import boxscoreadvancedv2

call = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id='0022200149')
player_stats = call.get_data_frames()[0]
team_stats = call.get_data_frames()[1]


# CumeStatsTeam

* game_by_game_stats:  
['JERSEY_NUM', 'PLAYER', 'PERSON_ID', 'TEAM_ID', 'GP', 'GS', 'ACTUAL_MINUTES', 'ACTUAL_SECONDS', 'FG', 'FGA', 'FG_PCT', 'FG3', 'FG3A', 'FG3_PCT', 'FT', 'FTA', 'FT_PCT', 'OFF_REB', 'DEF_REB', 'TOT_REB', 'AST', 'PF', 'DQ', 'STL', 'TURNOVERS', 'BLK', 'PTS', 'MAX_ACTUAL_MINUTES', 'MAX_ACTUAL_SECONDS', 'MAX_REB', 'MAX_AST', 'MAX_STL', 'MAX_TURNOVERS', 'MAX_BLKP', 'MAX_PTS', 'AVG_ACTUAL_MINUTES', 'AVG_ACTUAL_SECONDS', 'AVG_REB', 'AVG_AST', 'AVG_STL', 'AVG_TURNOVERS', 'AVG_BLKP', 'AVG_PTS', 'PER_MIN_REB', 'PER_MIN_AST', 'PER_MIN_STL', 'PER_MIN_TURNOVERS', 'PER_MIN_BLK', 'PER_MIN_PTS']

* total_team_stats:  
['CITY', 'NICKNAME', 'TEAM_ID', 'W', 'L', 'W_HOME', 'L_HOME', 'W_ROAD', 'L_ROAD', 'TEAM_TURNOVERS', 'TEAM_REBOUNDS', 'GP', 'GS', 'ACTUAL_MINUTES', 'ACTUAL_SECONDS', 'FG', 'FGA', 'FG_PCT', 'FG3', 'FG3A', 'FG3_PCT', 'FT', 'FTA', 'FT_PCT', 'OFF_REB', 'DEF_REB', 'TOT_REB', 'AST', 'PF', 'STL', 'TOTAL_TURNOVERS', 'BLK', 'PTS', 'AVG_REB', 'AVG_PTS', 'DQ']

In [43]:
from nba_api.stats.endpoints import cumestatsteam

call = cumestatsteam.CumeStatsTeam(game_ids='0022200149',team_id='1610612749')

game_by_game_stats = call.get_data_frames()[0]
test = call.get_data_frames()[1]
test.head()

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,TEAM_REBOUNDS,GP,GS,ACTUAL_MINUTES,ACTUAL_SECONDS,FG,FGA,FG_PCT,FG3,FG3A,FG3_PCT,FT,FTA,FT_PCT,OFF_REB,DEF_REB,TOT_REB,AST,PF,STL,TOTAL_TURNOVERS,BLK,PTS,AVG_REB,AVG_PTS,DQ
0,Milwaukee,Bucks,1610612749,0,1,0,0,0,1,1,12,1,5,240,0,36,87,0.414,13,40,0.325,13,16,0.813,6,35,53,21,18,5,19,9,98,53.0,98.0,0
1,OPPONENTS,,0,1,0,1,0,0,0,0,9,1,5,240,0,46,99,0.465,11,29,0.379,14,17,0.824,12,36,57,23,18,11,12,3,117,57.0,117.0,0


# CumeStatsTeamGames
Get a list of game_id for a team in a given season

* cume_stats_team_games: ['MATCHUP', 'GAME_ID']


In [13]:
from nba_api.stats.endpoints import cumestatsteamgames

call = cumestatsteamgames.CumeStatsTeamGames(team_id='1610612749',season=s)
cume_stats_team_games = call.get_data_frames()[0]
cume_stats_team_games.head()


,MATCHUP,GAME_ID
0,11/16/2022 Cavaliers at Bucks,0022200216
1,11/14/2022 Hawks at Bucks,0022200204
2,11/11/2022 Bucks at Spurs,0022200180
3,11/09/2022 Bucks at Thunder,0022200168
4,11/07/2022 Bucks at Hawks,0022200149


# MatchupsRollup

* matchups_rollup:  
['SEASON_ID', 'POSITION', 'PERCENT_OF_TIME', 'DEF_PLAYER_ID', 'DEF_PLAYER_NAME', 'GP', 'MATCHUP_MIN', 'PARTIAL_POSS', 'PLAYER_PTS', 'TEAM_PTS', 'MATCHUP_AST', 'MATCHUP_TOV', 'MATCHUP_BLK', 'MATCHUP_FGM', 'MATCHUP_FGA', 'MATCHUP_FG_PCT', 'MATCHUP_FG3M', 'MATCHUP_FG3A', 'MATCHUP_FG3_PCT', 'MATCHUP_FTM', 'MATCHUP_FTA', 'SFL']

In [6]:
from nba_api.stats.endpoints import matchupsrollup

call =  matchupsrollup.MatchupsRollup()
matchups_rollup = call.get_data_frames()[0]
matchups_rollup.head()

,SEASON_ID,POSITION,PERCENT_OF_TIME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,MATCHUP_AST,MATCHUP_TOV,MATCHUP_BLK,MATCHUP_FGM,MATCHUP_FGA,MATCHUP_FG_PCT,MATCHUP_FG3M,MATCHUP_FG3A,MATCHUP_FG3_PCT,MATCHUP_FTM,MATCHUP_FTA,SFL
0,22022,F,0.672738,2544,LeBron James,10,95:47,483.4,83,520,19,7,3,31,59,0.525,14,26,0.538,7,10,6
1,22022,G,0.217624,2544,LeBron James,10,30:59,178.9,32,190,10,6,2,13,36,0.361,5,17,0.294,1,2,1
2,22022,TOTAL,1.000000,2544,LeBron James,10,142:23,754.5,131,805,35,18,5,52,109,0.477,19,44,0.432,8,14,8
3,22022,C,0.109637,2544,LeBron James,10,15:37,92.2,16,95,6,5,0,8,14,0.571,0,1,0.000,0,2,1
4,22022,F,0.725498,2617,Udonis Haslem,2,3:06,16.9,4,21,1,0,1,2,4,0.500,0,0,0.000,0,0,0


# LeagueDashLineups

* lineups:  
['GROUP_SET', 'GROUP_ID', 'GROUP_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK']

In [10]:
from nba_api.stats.endpoints import leaguedashlineups

call =  leaguedashlineups.LeagueDashLineups()
lineups = call.get_data_frames()[0]
lineups.head()


,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-203991-1627749-1628381-1629027-1629631-,C. Capela - D. Murray - J. Collins - T. Young ...,1610612737,ATL,14,8,6,0.571,254.553333,245,506,0.484,49,153,0.320,80,100,0.800,61,193,254,138,62.0,33,28,28,88,109,619,35.0,1,1,4225,1783,1,1,1,1890,3,2,1586,2,2,1156,1,1,1,1,4237,1,1,4238,4238,1,1,14
1,Lineups,-202711-1630180-1630191-1630595-1631093-,B. Bogdanovic - S. Bey - I. Stewart - C. Cunni...,1610612765,DET,11,3,8,0.273,214.728333,179,401,0.446,59,163,0.362,109,126,0.865,52,155,207,105,64.0,29,14,38,93,104,526,-30.0,7,101,4237,2346,2,2,2,2030,2,1,1325,1,1,1059,2,2,2,4,4238,3,8,4239,4239,2,2,4231
2,Lineups,-203497-1626156-1626157-1630162-1630183-,R. Gobert - D. Russell - K. Towns - A. Edwards...,1610612750,MIN,13,6,7,0.462,197.603333,172,354,0.486,38,117,0.325,62,89,0.697,48,152,200,100,73.0,28,26,15,79,71,444,2.0,2,12,4231,2196,3,4,3,1886,7,5,1584,3,3,1430,3,3,3,5,4239,4,2,4236,4237,3,4,1034
3,Lineups,-201939-202691-203110-203952-1626172-,S. Curry - K. Thompson - D. Green - A. Wiggins...,1610612744,GSW,12,5,7,0.417,165.468333,173,318,0.544,67,140,0.479,54,63,0.857,27,129,156,132,47.0,25,14,11,70,63,467,77.0,5,30,4231,2204,4,3,4,1195,1,3,1087,4,4,1061,8,4,4,2,4235,6,8,4222,4235,4,3,1
4,Lineups,-201144-203482-203903-1628374-1629020-,M. Conley - K. Olynyk - J. Clarkson - L. Markk...,1610612762,UTA,13,7,6,0.538,153.793333,128,275,0.465,49,134,0.366,45,58,0.776,42,104,146,96,60.0,22,12,14,71,54,350,2.0,2,4,4225,1794,5,6,6,1944,3,4,1314,8,9,1236,4,6,5,6,4236,8,13,4232,4236,9,6,1034


# LeagueStandings

* leaugestandings:  
['LeagueID', 'SeasonID', 'TeamID', 'TeamCity', 'TeamName', 'Conference', 'ConferenceRecord', 'PlayoffRank', 'ClinchIndicator', 'Division', 'DivisionRecord', 'DivisionRank', 'WINS', 'LOSSES', 'WinPCT', 'LeagueRank', 'Record', 'HOME', 'ROAD', 'L10', 'Last10Home', 'Last10Road', 'OT', 'ThreePTSOrLess', 'TenPTSOrMore', 'LongHomeStreak', 'strLongHomeStreak', 'LongRoadStreak', 'strLongRoadStreak', 'LongWinStreak', 'LongLossStreak', 'CurrentHomeStreak', 'strCurrentHomeStreak', 'CurrentRoadStreak', 'strCurrentRoadStreak', 'CurrentStreak', 'strCurrentStreak', 'ConferenceGamesBack', 'DivisionGamesBack', 'ClinchedConferenceTitle', 'ClinchedDivisionTitle', 'ClinchedPlayoffBirth', 'EliminatedConference', 'EliminatedDivision', 'AheadAtHalf', 'BehindAtHalf', 'TiedAtHalf', 'AheadAtThird', 'BehindAtThird', 'TiedAtThird', 'Score100PTS', 'OppScore100PTS', 'OppOver500', 'LeadInFGPCT', 'LeadInReb', 'FewerTurnovers', 'PointsPG', 'OppPointsPG', 'DiffPointsPG', 'vsEast', 'vsAtlantic', 'vsCentral', 'vsSoutheast', 'vsWest', 'vsNorthwest', 'vsPacific', 'vsSouthwest', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'PreAS', 'PostAS']

In [12]:
from nba_api.stats.endpoints import leaguestandings

call =  leaguestandings.LeagueStandings(season=s)
leaugestandings = call.get_data_frames()[0]
leaugestandings.head()

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,DivisionRecord,DivisionRank,WINS,LOSSES,WinPCT,LeagueRank,Record,HOME,ROAD,L10,Last10Home,Last10Road,OT,ThreePTSOrLess,TenPTSOrMore,LongHomeStreak,strLongHomeStreak,LongRoadStreak,strLongRoadStreak,LongWinStreak,LongLossStreak,CurrentHomeStreak,strCurrentHomeStreak,CurrentRoadStreak,strCurrentRoadStreak,CurrentStreak,strCurrentStreak,ConferenceGamesBack,DivisionGamesBack,ClinchedConferenceTitle,ClinchedDivisionTitle,ClinchedPlayoffBirth,EliminatedConference,EliminatedDivision,AheadAtHalf,BehindAtHalf,TiedAtHalf,AheadAtThird,BehindAtThird,TiedAtThird,Score100PTS,OppScore100PTS,OppOver500,LeadInFGPCT,LeadInReb,FewerTurnovers,PointsPG,OppPointsPG,DiffPointsPG,vsEast,vsAtlantic,vsCentral,vsSoutheast,vsWest,vsNorthwest,vsPacific,vsSouthwest,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PreAS,PostAS
0,00,22022,1610612738,Boston,Celtics,East,9-3,1,,Atlantic,2-0,1,12,3,0.800,0,12-3,6-1,6-2,9-1,6-1,6-2,0-2,1-1,5-1,5,W 5,4,W 4,8,2,5,W 5,4,W 4,8,W 8,0.0,0.0,0,0,0,0,0,8-2,2-1,2-0,10-1,2-2,0-0,12-3,11-3,7-2,10-0,5-1,6-1,120.4,113.3,7.1,9-3,2-0,3-3,4-0,3-0,2-0,0-0,1-0,None,None,None,None,None,None,None,None,None,4-2,8-1,None,12-3,None
1,00,22022,1610612757,Portland,Trail Blazers,West,8-3,1,,Northwest,1-0,1,10,4,0.714,0,10-4,4-2,6-2,6-4,4-2,6-2,1-0,4-0,4-2,2,W 2,3,W 3,4,1,1,W 1,-1,L 1,1,W 1,0.0,0.0,0,0,0,0,0,5-0,5-4,0-0,6-1,3-3,1-0,10-2,8-4,5-3,7-0,10-1,4-0,109.9,107.6,2.3,2-1,0-0,0-0,2-1,8-3,1-0,4-1,3-2,None,None,None,None,None,None,None,None,None,5-1,5-3,None,10-4,None
2,00,22022,1610612749,Milwaukee,Bucks,East,7-2,2,,Central,3-0,1,11,3,0.786,0,11-3,8-1,3-2,7-3,8-1,3-2,1-0,2-0,7-3,7,W 7,2,W 2,9,2,1,W 1,-1,L 1,1,W 1,0.5,0.0,0,0,0,0,0,9-1,1-2,1-0,10-0,1-3,0-0,10-1,6-3,5-2,7-0,9-1,4-1,111.6,106.4,5.2,7-2,3-0,3-0,1-2,4-1,3-0,0-0,1-1,None,None,None,None,None,None,None,None,None,6-0,5-3,None,11-3,None
3,00,22022,1610612756,Phoenix,Suns,West,9-2,2,,Pacific,3-0,1,9,5,0.643,0,9-5,7-1,2-4,6-4,7-1,2-4,0-1,1-3,7-2,5,W 5,-2,L 2,5,2,2,W 2,-2,L 2,1,W 1,1.0,0.0,0,0,0,0,0,8-1,1-4,0-0,8-2,1-3,0-0,9-3,7-5,6-2,8-3,5-1,8-5,113.7,107.0,6.7,0-3,0-1,0-0,0-2,9-2,3-2,3-0,3-0,None,None,None,None,None,None,None,None,None,5-1,4-4,None,9-5,None
4,00,22022,1610612737,Atlanta,Hawks,East,7-5,3,,Southeast,1-1,1,9,6,0.600,0,9-6,5-3,4-3,5-5,5-3,4-3,1-0,1-0,6-4,2,W 2,2,W 2,3,2,-1,L 1,1,W 1,-1,L 1,3.0,0.0,0,0,0,0,0,6-0,3-6,0-0,7-1,1-5,1-0,9-6,5-6,5-5,9-0,5-2,8-3,114.6,113.9,0.7,7-5,2-3,4-1,1-1,2-1,0-1,0-0,2-0,None,None,None,None,None,None,None,None,None,4-3,5-3,None,9-6,None


# TeamDashboardByOpponent

* conference_team_dashboard:  
['GROUP_SET', 'GROUP_VALUE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'CFID', 'CFPARAMS']

In [33]:
from nba_api.stats.endpoints import teamdashboardbyopponent

call = teamdashboardbyopponent.TeamDashboardByOpponent(team_id=1610612742)#,date_to_nullable='2022-10-19'
overall_team_dashboard = call.get_data_frames()[0]
conference_team_dashboard = call.get_data_frames()[1]
division_team_dashboard = call.get_data_frames()[2]
opponent_team_dashboard = call.get_data_frames()[3]

#overall_team_dashboard.head()
#conference_team_dashboard.head()
#division_team_dashboard.head()
opponent_team_dashboard.head()

,GROUP_SET,GROUP_VALUE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
0,vs. Opponent,Brooklyn Nets,2,2,0,1.0,101.0,73,153,0.477,32,68,0.471,47,59,0.797,19,59,78,43,40.0,15,7,11,43,54,225,6.0,1,1,1,1,1,1,2,6,1,2,1,1,1,5,1,1,1,1,12,2,1,10,12,1,1,2,166,Brooklyn Nets
1,vs. Opponent,Houston Rockets,1,0,1,0.0,48.0,31,102,0.304,12,55,0.218,18,19,0.947,13,25,38,21,6.0,9,2,19,19,19,92,-9.0,3,8,7,8,4,12,3,12,8,3,11,8,11,1,3,11,8,6,1,5,9,12,4,11,12,12,166,Houston Rockets
2,vs. Opponent,LA Clippers,1,1,0,1.0,48.0,35,80,0.438,17,43,0.395,16,24,0.667,8,26,34,20,12.0,12,1,2,20,27,103,2.0,3,2,1,1,4,10,10,8,4,5,4,10,8,9,7,9,11,7,4,3,11,2,5,6,10,5,166,LA Clippers
3,vs. Opponent,Memphis Grizzlies,1,1,0,1.0,48.0,45,90,0.500,17,39,0.436,30,36,0.833,15,37,52,23,15.0,7,7,8,24,29,137,41.0,3,2,1,1,4,3,5,3,4,8,2,3,3,3,2,3,3,4,9,11,1,9,8,3,3,1,166,Memphis Grizzlies
4,vs. Opponent,New Orleans Pelicans,1,0,1,0.0,48.0,41,82,0.500,12,39,0.308,17,24,0.708,11,26,37,18,12.0,8,5,3,16,21,111,-2.0,3,8,7,8,4,5,8,3,8,8,8,9,8,7,5,9,9,10,4,8,3,4,3,8,5,8,166,New Orleans Pelicans


# TeamGameStreakFinder

* team_game_streak_finder_parameters_results:  
['TEAM_NAME', 'TEAM_ID', 'GAMESTREAK', 'STARTDATE', 'ENDDATE', 'ACTIVESTREAK', 'NUMSEASONS', 'LASTSEASON', 'FIRSTSEASON', 'ABBREVIATION']

In [36]:
from nba_api.stats.endpoints import teamgamestreakfinder

call = teamgamestreakfinder.TeamGameStreakFinder( season_nullable= s)
team_game_streak_finder_parameters_results = call.get_data_frames()[0]
team_game_streak_finder_parameters_results.head(50)

,TEAM_NAME,TEAM_ID,GAMESTREAK,STARTDATE,ENDDATE,ACTIVESTREAK,NUMSEASONS,LASTSEASON,FIRSTSEASON,ABBREVIATION
0,Toronto Raptors,1610612761,16,2022-10-19T00:00:00,2022-11-16T00:00:00,1,1,2022-23,2022-23,None
1,Charlotte Hornets,1610612766,16,2022-10-19T00:00:00,2022-11-16T00:00:00,1,1,2022-23,2022-23,None
2,Utah Jazz,1610612762,16,2022-10-19T00:00:00,2022-11-15T00:00:00,1,1,2022-23,2022-23,None
3,Golden State Warriors,1610612744,15,2022-10-18T00:00:00,2022-11-16T00:00:00,1,1,2022-23,2022-23,None
4,Orlando Magic,1610612753,15,2022-10-19T00:00:00,2022-11-16T00:00:00,1,1,2022-23,2022-23,None
5,Atlanta Hawks,1610612737,15,2022-10-19T00:00:00,2022-11-16T00:00:00,1,1,2022-23,2022-23,None
6,Houston Rockets,1610612745,15,2022-10-19T00:00:00,2022-11-16T00:00:00,1,1,2022-23,2022-23,None
7,New Orleans Pelicans,1610612740,15,2022-10-19T00:00:00,2022-11-16T00:00:00,1,1,2022-23,2022-23,None
8,Chicago Bulls,1610612741,15,2022-10-19T00:00:00,2022-11-16T00:00:00,1,1,2022-23,2022-23,None
9,Miami Heat,1610612748,15,2022-10-19T00:00:00,2022-11-16T00:00:00,1,1,2022-23,2022-23,None
